# 1. Create FRCNN Model

Title: Faster-RCNN

Author: [Blaine Perry]

Date created: 2021/08/03

Last modified: 2021/08/03

Description: Implementation of Faster-RCNN.

## Train the model

# Start by creating the workspace

In [1]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azure-mgmt-resource 18.0.0 (/anaconda/envs/azureml_py38/lib/python3.8/site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 4.0.1 (/anaconda/envs/azureml_py38/lib/python3.8/site-packages), Requirement.parse('pyarrow<4.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azure-mgmt-resource 18.0.0 (/anaconda/envs/azureml_py38/lib/python3.8/site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed t

Azure ML SDK Version:  1.31.0


In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: geointerns_workspace
Azure region: usgovarizona
Subscription id: 8dab0d11-5037-44d3-9b24-ed18e6c0845e
Resource group: iso_lrpf


## Create an experiment

In [4]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='keras-FRCNN')

## Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. Here we wil create `AmlCompute` as your training compute resource.


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "two-gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC12', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-08-10T15:15:49.295000+00:00', 'errors': None, 'creationTime': '2021-07-08T18:36:44.988863+00:00', 'modifiedTime': '2021-07-08T18:37:15.441038+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC12'}


## Create an environment

Define a conda environment YAML file with your training script dependencies, which include TensorFlow, Keras and matplotlib, and create an Azure ML environment.

In [6]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.8.8
- pip:
  - azureml-defaults
  - tensorflow-gpu==2.5


Overwriting conda_dependencies.yml


In [7]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

# Specify a GPU base image
keras_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04:20210615.v1'

## Configure the training job

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

Note that we are specifying a DatasetConsumptionConfig for our FileDataset as an argument to the training script. Azure ML will resolve this DatasetConsumptionConfig to the mount-point of the backing datastore, which we access from the training script.

In [8]:
from azureml.core import Dataset
dataset = Dataset.get_by_name(ws, 'xview_tfrecords')

# list the files referenced by the dataset
dataset.to_path()


['/xview_class_labels.txt',
 '/xview_test_sample.record',
 '/xview_train_sample.record']

In [9]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', dataset.as_named_input('xview_tfrecords').as_mount(),
        '--num-epochs', 30]

src = ScriptRunConfig(source_directory='./FRCNN',
                      script='train_frcnn.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=keras_env)

In [10]:
run = exp.submit(src)